# Model project


For the model project, we've decided to formulate the OLG model in python and solve it through the tools and scope of this course. To do so, we adjust the parameters of the model and in this case increase population growth and see which effects it has

The model will include the following:
    1. Packages
    2. Model description 
    3. Analytical solution
    4. Numerical solution
    5. Concluding remarks 

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm
from types import SimpleNamespace

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

import ipywidgets as widgets

import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# Model description

We consider a two period OLG model where we include an Euler equation.

**Firms maximization problem:**
$$
\max_{K_t, L_t}  AK_t^{\alpha}L_t^{1-\alpha} - r_t K_t - w_t L_t
$$
$L_t$ and $K_t$ are the labor and capital employed by a firm. $A$ is the total factor productivity and must be greater than 0.
$\alpha \in (0,1)$ and is the percentage of capital the consumer uses in their production process.

**Production function:**
We use a Cobb Douglas production function:
$$
Y_t = AK_t^{\alpha}L_t^{1-\alpha} 
$$
$L_t$ shows how many individuals are born in period t. The population grows at a constant rate, $n$:
$$
L_t = L_{t-1}(1+n)
$$

The capital accumulation in the next period is equal to the savings from this period:
$$
K_{t+1} = S_t = s_tL_t
$$
It is only the young who save.

**Capital per capita:**
$$
k_t = \frac{K_t}{L_t}
$$

In period $t+1$ aggregate capital stock is equal to what the young people save, what the old people dissave and un-depreciated capital ($\delta = 0$):
$$
k_{t+1}(1+n) = s_t
$$

**Saving problem for an individual:**
$$
U_t = \ln(c_{1t})+\beta \ln(c_{2t+1})
$$
$c_{1t}$ is consumption when young and $c_{2t+1}$ is consumption when old.

**Parameter expression:**
$$
\beta = \frac{1}{1+\rho}
$$
$\rho$ is the discount rate and must be larger than $-1$, such that $\beta>0$.

**Budget constraints:**
$$
c_{1t} + s_t = w_t 
$$

$$
c_{2t+1} = (1 + r_{t+1}) s_t
$$
$w_t$ is defined as the wage rate and $r_{t+1}$ is the interest rate between $t$ and $t+1$.

The consumption for young is equal to their wage minus savings. Since old agents do not work, they must provide consumption through savings which are remunerated at the gross rate, $(1 + r_{t+1})$.  

**Lifetime budget constraint:**
By combining consumption when young and consumption when old, we yield the lifetime budget constraint:
$$
c_{1t} + \frac{c_{2t+1}}{1+r_{t+1}} = w_t 
$$
The old cannot make any choices and must therefore consume their wealth.

We assume $\delta = 0$ meaning zero depreciation. By solving the firms' maximization problem we get factor prices for $r_t$ and $w_t$:
$$
r_t = A \alpha k_t^{\alpha-1}
$$

$$
w_t = A(1-\alpha)k_t^\alpha
$$

In order to derive the Euler equation, i.e., solving the household problem, we use the Lagrangian to find FOC's. Then we can substitute out the Lagrangian multiplier to get the Euler equation.

**The Euler equation:**
$$
c_{2t+1} = \frac{1+r_{t+1}}{1+\rho}c_{1t}
$$

By combining the budget constraints with the Euler equation, we are able to find the optimal savings. 
$$
s_t = \frac{1}{2+\rho}w_t
$$

Remember that $k_{t+1} (1+n) = s_t$. By inserting the factor price for wage, we get the capital accumulation per capita. 

**Capital accumulation per capita:**
$$
k_{t+1}(1+n) = \frac{1}{2+\rho}w_t
$$

$$
k_{t+1}(1+n) = \frac{1}{2+\rho}A(1-\alpha)k_t^\alpha
$$

$$
k_{t+1} = \frac{A(1-\alpha)k_t^\alpha}{(1+n)(2+\rho)}
$$

In steady state $k_{t+1} = k_t = k^*$ yielding:

**Steady state:**
$$
k^* = \left[\frac{A(1-\alpha)}{(1+n)(2+\rho)}\right]^\frac{1}{1-\alpha}


## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

We start by assigning the parameters values:

In [2]:
alpha = sm.symbols('alpha')
rho =  sm.symbols('rho')
k = sm.symbols('k')
n = sm.symbols('n')
A = sm.symbols('A')

Next, we solve for steady state 

In [3]:
ss = sm.Eq(k, (A*(1-alpha)*k**alpha)/((1+n)*(2+rho)))
ss

Eq(k, A*k**alpha*(1 - alpha)/((n + 1)*(rho + 2)))

And convert the function to python

In [4]:
ss_solve = sm.solve(ss, k)[0]
ss_solve

(A*(1 - alpha)/(n*rho + 2*n + rho + 2))**(-1/(alpha - 1))

Next, we solve for k in ss

In [5]:
k_solve = sm.lambdify((alpha,rho,n,A),ss_solve)
k_solve(1/3,0.05,0.04,1)

0.17485671329443958

# Numerical solution

Now we wish to solve the problem numerically and start by defining the values 

In [6]:
rho = 0.05
A = 1.00
alpha = 1/3
n = 0.04
tau = 0.00


ks_1 = ((A*(1-alpha))/((1+n)*(2+rho)))**(-1/(alpha-1))



Next, we define the objective while simoultaneously solving for the roots of the problem 

In [7]:
obj = lambda k: k - ((A*(1-alpha)*k**alpha)/((1+n)*(2+rho)))


result = optimize.root_scalar(obj,bracket=[1e-10,10],method='bisect') 
result.root

0.17485671329430136

Now we set up the simulation


In [ ]:
from modelproject import modelclass

model = modelclass()
par = model.par
sim = model.sim

model.sim_results()

Finally, we call on out sim and make use of the new variables and plot the results

In [ ]:
 
model.simulate()


fig = plt.figure(figsize=(6,6/1.5))
ax = fig.add_subplot(1,1,1)
ax.plot(model.sim.k,label=r'$k_{t}$, (n = 0.05)') # Plotting k
ax.axhline(ks_1,ls='--',color='black',label='analytical steady state') # Plotting the analytical steady state
ax.legend(frameon=True,fontsize=12)
ax.set_title('Convergence of capital accumulation')
ax.set_xlabel('Numbers of periods')
ax.set_ylabel('Steady state value')
fig.tight_layout()
sim_k_old = model.sim.k.copy() # Creating a copy of the steady state simulation for later

# Conclusion


In summary, the initial figure indicates that the analytical steady state capital accumulation is around 0.175 when there are no external shocks. However, if there is an unexpected increase in government spending, funded by lump-sum taxes imposed on the young workers each period, the capital accumulation correspondingly decreases.